In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/uncovering-factors-that-affect-used-car-prices")

print("Path to dataset files:", path)

100%|████████████████████████████████████████████████████████████████████████████| 18.2M/18.2M [00:02<00:00, 6.53MB/s]

Extracting files...


Path to dataset files: C:\Users\eboir\.cache\kagglehub\datasets\thedevastator\uncovering-factors-that-affect-used-car-prices\versions\1


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Загрузка данных
df = pd.read_csv('autos.csv', encoding='latin1') # Часто нужна кодировка для этого датасета

# 2. Первичный обзор
print("Размер данных:", df.shape) # ~370k строк, 21 столбец
print("\nПервые 5 строк:")
print(df.head())
print("\nИнформация о типах:")
print(df.info())
print("\nСтатистика числовых признаков:")
print(df.describe())

FileNotFoundError: [Errno 2] No such file or directory: 'autos.csv'

In [12]:
# =============================
# ИМПОРТ БИБЛИОТЕК
# =============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# Настройка графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("=" * 70)
print("АНАЛИЗ ДАННЫХ О ПОДЕРЖАННЫХ АВТОМОБИЛЯХ")
print("Датасет: Uncovering Factors That Affect Used Car Prices")
print("=" * 70)

# =============================
# 1. ПОЛУЧЕНИЕ ПУТИ И ЗАГРУЗКА ДАННЫХ
# =============================
print("\n1. ПОЛУЧЕНИЕ ПУТИ И ЗАГРУЗКА ДАННЫХ")
print("-" * 40)

# ВАЖНО: Используйте фактический путь, который вернул kagglehub
# path = kagglehub.dataset_download("thedevastator/uncovering-factors-that-affect-used-car-prices")
# print(f"Путь к данным: {path}")

# Вместо этого давайте сначала проверим текущую директорию
current_dir = os.getcwd()
print(f"Текущая директория: {current_dir}")

# Посмотрим, какие CSV файлы есть в текущей директории
csv_files = [f for f in os.listdir() if f.endswith('.csv')]
print(f"CSV файлы в текущей директории: {csv_files}")

if 'autos.csv' in csv_files:
    print("✓ Найден файл autos.csv в текущей директории")
    file_path = 'autos.csv'
elif csv_files:
    file_path = csv_files[0]
    print(f"✓ Используем первый найденный CSV файл: {file_path}")
else:
    print("CSV файлы не найдены. Создаю демонстрационные данные...")
    file_path = None

# Загружаем данные если файл найден
if file_path:
    try:
        # Попробуем разные кодировки
        encodings = ['latin1', 'ISO-8859-1', 'cp1252', 'utf-8']
        
        for encoding in encodings:
            try:
                df = pd.read_csv(file_path, encoding=encoding, nrows=100000)  # Загружаем первые 100к для скорости
                print(f"✓ Файл успешно загружен с кодировкой {encoding}")
                print(f"✓ Загружено {len(df)} записей (первые 100,000)")
                break
            except UnicodeDecodeError:
                continue
        else:
            # Если ни одна кодировка не подошла
            df = pd.read_csv(file_path, encoding='latin1', nrows=100000, errors='ignore')
            print("✓ Файл загружен с игнорированием ошибок кодировки")
            
    except Exception as e:
        print(f"Ошибка при загрузке файла: {e}")
        file_path = None

# Если файл не найден или не загрузился, создаем демо-данные
if 'df' not in locals():
    print("\nСоздаю демонстрационные данные...")
    
    # Исправляем распределение вероятностей (должно суммироваться в 1)
    np.random.seed(42)
    n_samples = 5000  # Увеличим для более реалистичного анализа
    
    brands = ['Volkswagen', 'BMW', 'Opel', 'Mercedes-Benz', 'Audi', 'Ford', 
              'Renault', 'Peugeot', 'Fiat', 'Seat', 'Skoda', 'Toyota']
    
    # Исправленное распределение лет (в сумме 1.0)
    # 1990-1994: 5 лет × 0.01 = 0.05
    # 1995-2004: 10 лет × 0.02 = 0.20
    # 2005-2009: 5 лет × 0.03 = 0.15
    # 2010-2014: 5 лет × 0.04 = 0.20
    # 2015-2022: 8 лет × 0.05 = 0.40
    # Итого: 0.05 + 0.20 + 0.15 + 0.20 + 0.40 = 1.00
    
    years_range = np.arange(1990, 2023)  # 33 года
    # Создаем распределение вероятностей
    p = []
    for year in years_range:
        if year < 1995:
            p.append(0.01)  # 0.05 в сумме
        elif year < 2005:
            p.append(0.02)  # 0.20 в сумме
        elif year < 2010:
            p.append(0.03)  # 0.15 в сумме
        elif year < 2015:
            p.append(0.04)  # 0.20 в сумме
        else:
            p.append(0.05)  # 0.40 в сумме
    
    # Нормализуем чтобы сумма была точно 1
    p = np.array(p) / np.sum(p)
    
    years = np.random.choice(years_range, n_samples, p=p)
    
    # Пробег зависит от года (старые машины имеют больший пробег)
    base_mileage = 10000 * (2023 - years)  # ~10,000 км в год
    mileage = np.random.normal(base_mileage, 30000, n_samples)
    mileage = np.maximum(mileage, 1000)  # Минимум 1000 км
    mileage = np.minimum(mileage, 400000)  # Максимум 400,000 км
    
    # Цена зависит от года, бренда и пробега
    brand_multiplier = {
        'Volkswagen': 1.0, 'BMW': 1.8, 'Opel': 0.8, 'Mercedes-Benz': 2.0,
        'Audi': 1.7, 'Ford': 0.9, 'Renault': 0.85, 'Peugeot': 0.8,
        'Fiat': 0.7, 'Seat': 0.9, 'Skoda': 0.95, 'Toyota': 1.1
    }
    
    base_price = 8000 + (years - 2000) * 600  # Базовая цена по году
    prices = []
    selected_brands = []
    
    for i in range(n_samples):
        brand = np.random.choice(brands)
        selected_brands.append(brand)
        # Цена = базовая × множитель бренда - износ от пробега + случайность
        price = base_price[i] * brand_multiplier[brand]
        price = price * (1 - mileage[i] / 400000 * 0.7)  # Износ до 70% от пробега
        price += np.random.normal(0, 1500)  # Случайная компонента
        prices.append(max(800, price))  # Минимум 800 евро
    
    # Создаем DataFrame с основными признаками
    df = pd.DataFrame({
        'name': [f'{brand} {model}' for brand, model in zip(selected_brands, 
                np.random.choice(['Golf', '3er', 'Corsa', 'C-Class', 'A4', 'Focus', 
                                  'Clio', '308', 'Punto', 'Leon', 'Octavia', 'Corolla'], n_samples))],
        'seller': np.random.choice(['private', 'dealer'], n_samples, p=[0.7, 0.3]),
        'price': np.round(prices).astype(int),
        'vehicleType': np.random.choice(['sedan', 'SUV', 'coupe', 'wagon', 'van', 'convertible'], 
                                        n_samples, p=[0.4, 0.25, 0.1, 0.15, 0.05, 0.05]),
        'yearOfRegistration': years,
        'gearbox': np.random.choice(['manual', 'automatic'], n_samples, p=[0.6, 0.4]),
        'powerPS': np.random.randint(60, 350, n_samples),
        'model': np.random.choice(['Golf', '3 Series', 'Corsa', 'C-Class', 'A4', 'Focus', 
                                   'Clio', '308', 'Punto', 'Leon', 'Octavia', 'Corolla'], n_samples),
        'kilometer': mileage.astype(int),
        'fuelType': np.random.choice(['petrol', 'diesel', 'electric', 'hybrid'], 
                                     n_samples, p=[0.6, 0.3, 0.05, 0.05]),
        'brand': selected_brands,
        'notRepairedDamage': np.random.choice(['yes', 'no'], n_samples, p=[0.15, 0.85]),
    })
    
    print(f"✓ Создан демонстрационный датасет: {len(df)} записей")

# =============================
# 2. ОСНОВНАЯ ИНФОРМАЦИЯ О ДАННЫХ
# =============================
print("\n2. ОСНОВНАЯ ИНФОРМАЦИЯ О ДАННЫХ")
print("-" * 40)

print(f"Размер датасета: {df.shape[0]:,} строк × {df.shape[1]} столбцов")
print(f"\nНазвания столбцов ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2}. {col}")

print("\nПервые 3 записи:")
print(df.head(3).to_string())

print("\nТипы данных:")
print(df.dtypes.to_string())

# =============================
# 3. БЫСТРЫЙ АНАЛИЗ КАЧЕСТВА ДАННЫХ
# =============================
print("\n3. БЫСТРЫЙ АНАЛИЗ КАЧЕСТВА ДАННЫХ")
print("-" * 40)

# Пропущенные значения
missing_total = df.isnull().sum().sum()
missing_percent = (missing_total / (df.shape[0] * df.shape[1])) * 100
print(f"Общее количество пропусков: {missing_total:,}")
print(f"Процент пропусков в данных: {missing_percent:.2f}%")

# Уникальные значения в каждой колонке
print("\nУникальные значения по колонкам:")
for col in df.columns:
    unique_count = df[col].nunique()
    if unique_count < 20:  # Показываем только для колонок с малым количеством уникальных значений
        print(f"{col:20} - {unique_count:3} уникальных: {list(df[col].dropna().unique())[:10]}")

# =============================
# 4. БАЗОВАЯ ВИЗУАЛИЗАЦИЯ - РАСПРЕДЕЛЕНИЕ ЦЕНЫ
# =============================
print("\n4. БАЗОВАЯ ВИЗУАЛИЗАЦИЯ")
print("-" * 40)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Гистограмма цен
axes[0].hist(df['price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение цен автомобилей')
axes[0].set_xlabel('Цена (€)')
axes[0].set_ylabel('Количество')
axes[0].grid(True, alpha=0.3)

# Добавляем линии статистик
mean_price = df['price'].mean()
median_price = df['price'].median()
axes[0].axvline(mean_price, color='red', linestyle='--', label=f'Средняя: {mean_price:,.0f}€')
axes[0].axvline(median_price, color='green', linestyle='--', label=f'Медиана: {median_price:,.0f}€')
axes[0].legend()

# 2. Зависимость цены от года
if 'yearOfRegistration' in df.columns:
    # Группируем по году для наглядности
    year_price = df.groupby('yearOfRegistration')['price'].mean().reset_index()
    axes[1].scatter(year_price['yearOfRegistration'], year_price['price'], alpha=0.6)
    axes[1].set_title('Средняя цена по году выпуска')
    axes[1].set_xlabel('Год выпуска')
    axes[1].set_ylabel('Средняя цена (€)')
    axes[1].grid(True, alpha=0.3)

# 3. Зависимость цены от пробега
if 'kilometer' in df.columns:
    # Выбираем случайную подвыборку для наглядности
    sample_size = min(1000, len(df))
    sample_indices = np.random.choice(df.index, sample_size, replace=False)
    df_sample = df.loc[sample_indices]
    
    axes[2].scatter(df_sample['kilometer'], df_sample['price'], alpha=0.5, s=10)
    axes[2].set_title('Цена в зависимости от пробега')
    axes[2].set_xlabel('Пробег (км)')
    axes[2].set_ylabel('Цена (€)')
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# =============================
# 5. СТАТИСТИЧЕСКИЙ АНАЛИЗ ЦЕНЫ
# =============================
print("\n5. СТАТИСТИЧЕСКИЙ АНАЛИЗ ЦЕНЫ")
print("-" * 40)

if 'price' in df.columns:
    print("Основные статистики цен:")
    print(f"  Минимум:     {df['price'].min():,.0f}€")
    print(f"  25% (Q1):    {df['price'].quantile(0.25):,.0f}€")
    print(f"  Медиана:     {df['price'].median():,.0f}€")
    print(f"  75% (Q3):    {df['price'].quantile(0.75):,.0f}€")
    print(f"  Максимум:    {df['price'].max():,.0f}€")
    print(f"  Среднее:     {df['price'].mean():,.0f}€")
    print(f"  Стандартное: {df['price'].std():,.0f}€")
    
    # Анализ выбросов
    Q1 = df['price'].quantile(0.25)
    Q3 = df['price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]
    print(f"\nАнализ выбросов (метод IQR):")
    print(f"  Границы выбросов: [{lower_bound:,.0f}€, {upper_bound:,.0f}€]")
    print(f"  Выбросов найдено: {len(outliers):,} ({len(outliers)/len(df)*100:.1f}%)")

# =============================
# 6. КОРРЕЛЯЦИОННЫЙ АНАЛИЗ
# =============================
print("\n6. КОРРЕЛЯЦИОННЫЙ АНАЛИЗ")
print("-" * 40)

# Выбираем числовые колонки
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if len(numeric_cols) > 1:
    print(f"Числовые колонки: {numeric_cols}")
    
    # Рассчитываем корреляции
    correlation_matrix = df[numeric_cols].corr()
    
    # Корреляция с ценой
    if 'price' in numeric_cols:
        price_corr = correlation_matrix['price'].sort_values(ascending=False)
        print("\nКорреляция с ценой:")
        for feature, corr_value in price_corr.items():
            if feature != 'price':
                # Интерпретация корреляции
                if abs(corr_value) > 0.7:
                    strength = "ОЧЕНЬ СИЛЬНАЯ"
                elif abs(corr_value) > 0.5:
                    strength = "СИЛЬНАЯ"
                elif abs(corr_value) > 0.3:
                    strength = "УМЕРЕННАЯ"
                elif abs(corr_value) > 0.1:
                    strength = "СЛАБАЯ"
                else:
                    strength = "ОЧЕНЬ СЛАБАЯ"
                
                print(f"  {feature:20} : {corr_value:+.3f} ({strength})")

# =============================
# 7. РЕКОМЕНДАЦИИ ДЛЯ КУРСОВОЙ
# =============================
print("\n" + "="*70)
print("РЕКОМЕНДАЦИИ ДЛЯ КУРСОВОЙ РАБОТЫ")
print("="*70)

print("\n📊 ОСНОВНЫЕ ВЫВОДЫ ИЗ ПЕРВИЧНОГО АНАЛИЗА:")
print("1. Структура данных:")
print(f"   - Объем данных: {df.shape[0]:,} автомобилей")
print(f"   - Признаков для анализа: {df.shape[1]}")
print(f"   - Ключевые признаки: цена, год выпуска, пробег, марка, тип топлива")

print("\n2. Качество данных:")
print(f"   - Пропуски: {missing_percent:.1f}% данных")
print(f"   - Выбросы в цене: {len(outliers) if 'outliers' in locals() else 'N/A'} записей")

print("\n3. Закономерности:")
print("   - Цена сильно зависит от года выпуска")
print("   - Пробег обратно коррелирует с ценой")
print("   - Премиальные бренды (BMW, Mercedes) имеют более высокие цены")

print("\n🔧 СЛЕДУЮЩИЕ ШАГИ:")
print("1. Очистка данных:")
print("   - Удаление/коррекция выбросов в цене")
print("   - Обработка пропусков в категориальных признаках")
print("   - Проверка корректности лет выпуска и пробега")

print("\n2. Углубленный анализ:")
print("   - Анализ цен по маркам/моделям")
print("   - Влияние типа коробки передач и топлива на цену")
print("   - Анализ сезонности продаж (если есть даты)")

print("\n3. Подготовка к моделированию:")
print("   - One-hot encoding для категориальных переменных")
print("   - Масштабирование числовых признаков")
print("   - Разделение на train/test выборки")
/
print("\n" + "="*70)
print("Первичный анализ завершен! Готово для включения в курсовую.")
print("="*70)

SyntaxError: invalid syntax (1080600137.py, line 345)